In [1]:
import pandas as pd
import warnings
from hyperimpute.utils.serialization import load_model_from_file, save_model_to_file
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

workspace = Path("workspace")
results_dir = Path("results")
data_dir = Path("data")

workspace.mkdir(parents=True, exist_ok=True)

warnings.filterwarnings("ignore")

cat_limit = 10
n_seeds = 5

version = "take6_v1"
changelog = f"multiple_imputation{n_seeds}_last_try"


In [2]:
def augment_base_dataset(df):
    for rid in df["RID_HASH"].unique():
        visits = len(df[df["RID_HASH"] == rid])
        last_visit = df[df["RID_HASH"] == rid]["VISCODE"].max()

        df.loc[df["RID_HASH"] == rid, "total_visits"] = visits
        df.loc[df["RID_HASH"] == rid, "last_visit"] = last_visit

    return df

def dataframe_hash(df: pd.DataFrame) -> str:
    cols = sorted(list(df.columns))
    return str(abs(pd.util.hash_pandas_object(df[cols].fillna(0)).sum()))


In [3]:
dev_set = pd.read_csv(data_dir / "dev_set.csv")
dev_set = dev_set.sort_values(["RID_HASH", "VISCODE"])
dev_set = augment_base_dataset(dev_set)

scaled_cols = [
    "MMSE",
    "ADAS13",
    "Ventricles",
    "Hippocampus",
    "WholeBrain",
    "Entorhinal",
    "Fusiform",
    "MidTemp",
]

scaler = MinMaxScaler().fit(dev_set[scaled_cols])
dev_set[scaled_cols] = scaler.transform(dev_set[scaled_cols])

dev_set

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
2163,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,0,79.1,0,20,1.0,1.0,0.5,0.923077,0.164384,0.071871,0.548646,0.376516,0.464021,0.194906,0.400709,2.0,6.0
154,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,6,79.6,0,20,1.0,1.0,1.5,0.923077,0.237397,0.071956,0.548307,0.366398,0.403880,0.193367,0.397291,2.0,6.0
1385,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,0,72.9,1,12,1.0,1.0,1.0,1.000000,0.123288,0.142655,0.525169,0.235599,0.513404,0.356253,0.294774,6.0,60.0
2698,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,6,73.4,1,12,1.0,1.0,1.0,1.000000,0.164384,0.144729,0.549210,0.230361,0.435097,0.322395,0.294175,6.0,60.0
2291,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,12,73.9,1,12,1.0,1.0,1.0,0.961538,0.109589,0.155550,0.527878,0.215944,0.487831,0.342600,0.277552,6.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,60,79.8,1,19,1.0,0.0,3.0,0.923077,0.223699,0.170895,0.357020,0.321346,0.310935,0.399047,0.461476,7.0,102.0
2646,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,102,83.3,1,19,1.0,0.0,3.0,0.846154,0.168904,0.178231,0.352043,0.309095,0.256790,0.372685,0.416478,7.0,102.0
1962,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,0,72.1,0,12,1.0,0.0,0.5,0.884615,0.150685,0.416382,0.602438,0.636654,0.610229,0.743037,0.624631,3.0,24.0
122,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,12,73.1,0,12,1.0,0.0,1.0,0.961538,0.155205,0.398451,0.608521,0.634650,0.617108,0.729087,0.638477,3.0,24.0


In [4]:
dev_1 = pd.read_csv(data_dir / "dev_1.csv")
dev_1 = dev_1.sort_values(["RID_HASH", "VISCODE"])
dev_1 = augment_base_dataset(dev_1)
dev_1[scaled_cols] = scaler.transform(dev_1[scaled_cols])

dev_1

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
2163,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,0,NaN,0.0,20.0,1.0,1.0,0.5,0.923077,0.164384,NaN,NaN,0.376516,NaN,NaN,NaN,2.0,6.0
154,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,6,79.6,0.0,20.0,1.0,1.0,1.5,0.923077,0.237397,0.071956,0.548307,0.366398,0.403880,0.193367,0.397291,2.0,6.0
1385,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,0,NaN,1.0,12.0,NaN,1.0,NaN,NaN,NaN,NaN,0.525169,0.235599,0.513404,0.356253,0.294774,6.0,60.0
2698,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,6,NaN,1.0,12.0,NaN,1.0,NaN,NaN,NaN,NaN,0.549210,0.230361,0.435097,0.322395,0.294175,6.0,60.0
2291,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,12,NaN,1.0,12.0,NaN,1.0,NaN,NaN,NaN,NaN,0.527878,0.215944,0.487831,0.342600,0.277552,6.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,60,79.8,1.0,19.0,NaN,0.0,NaN,NaN,NaN,0.170895,NaN,0.321346,NaN,NaN,NaN,7.0,102.0
2646,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,102,83.3,1.0,19.0,NaN,0.0,NaN,NaN,NaN,0.178231,NaN,0.309095,NaN,NaN,NaN,7.0,102.0
1962,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,0,72.1,NaN,12.0,1.0,0.0,0.5,0.884615,0.150685,0.416382,0.602438,NaN,0.610229,0.743037,0.624631,3.0,24.0
122,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,12,73.1,NaN,12.0,1.0,0.0,1.0,0.961538,0.155205,0.398451,0.608521,NaN,0.617108,0.729087,0.638477,3.0,24.0


In [5]:
dev_2 = pd.read_csv(data_dir / "dev_2.csv")
dev_2 = dev_2.sort_values(["RID_HASH", "VISCODE"])
dev_2 = augment_base_dataset(dev_2)
dev_2[scaled_cols] = scaler.transform(dev_2[scaled_cols])

dev_2

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
2163,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,0,79.1,0.0,20.0,1.0,1.0,0.5,0.923077,0.164384,0.071871,0.548646,0.376516,0.464021,0.194906,0.400709,2.0,6.0
154,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,6,79.6,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.071956,0.548307,NaN,0.403880,0.193367,0.397291,2.0,6.0
1385,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,0,72.9,NaN,12.0,1.0,1.0,1.0,1.000000,0.123288,0.142655,0.525169,NaN,0.513404,0.356253,0.294774,6.0,60.0
2698,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,6,NaN,NaN,12.0,1.0,1.0,1.0,1.000000,0.164384,NaN,0.549210,NaN,0.435097,0.322395,0.294175,6.0,60.0
2291,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,12,NaN,NaN,12.0,1.0,1.0,1.0,0.961538,0.109589,NaN,NaN,NaN,NaN,NaN,NaN,6.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,60,NaN,NaN,19.0,1.0,0.0,3.0,0.923077,0.223699,NaN,0.357020,NaN,0.310935,0.399047,0.461476,7.0,102.0
2646,ff59785f0d6b12fc51a07f09bb3a02790e54d04bb0803b...,102,NaN,NaN,19.0,1.0,0.0,3.0,0.846154,0.168904,NaN,0.352043,NaN,0.256790,0.372685,0.416478,7.0,102.0
1962,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,0,72.1,NaN,12.0,NaN,0.0,NaN,NaN,NaN,0.416382,0.602438,NaN,0.610229,0.743037,0.624631,3.0,24.0
122,ff98c50c3e97b776ab61db883cf1c8fd5a6d304d7165c8...,12,NaN,NaN,12.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,24.0


In [6]:
submission = pd.read_csv(data_dir / "sample_submission.csv")

submission.values[1]

array(['6b6a7136f42a8dbd469a201b88e2abb54a93667822761357db2f6d620da6af8a_0_Ventricles_test_A',
       40613.0818580834], dtype=object)

In [7]:
test_A = pd.read_csv(data_dir / "test_A.csv")
test_A = augment_base_dataset(test_A)
test_A[scaled_cols] = scaler.transform(test_A[scaled_cols])

test_A

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
0,988b6137f4352c01e4b52790505caa0c3ec438f117000a...,24,NaN,NaN,18.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,36.0
1,fb640cef87a6af00053e632140ce18f5722431bb92576b...,12,66.4,1.0,18.0,1.0,1.0,1.5,0.961538,0.077671,0.145063,NaN,0.542904,NaN,NaN,NaN,5.0,24.0
2,f24f78d62c90319b575dfb48a482159c4d0df14cb71530...,66,74.5,0.0,14.0,0.0,0.0,0.0,0.961538,0.050274,0.559104,0.565102,0.753302,0.641093,0.911086,0.866886,5.0,96.0
3,da4cbd3f09e8ddc87cc72e542d43f072e7df288face65e...,0,NaN,NaN,16.0,0.0,0.0,0.0,1.000000,0.191781,NaN,NaN,NaN,NaN,NaN,NaN,2.0,36.0
4,f665c6ee86356bdd135be03c61348607cabd64ed8433ba...,12,82.7,NaN,13.0,NaN,1.0,NaN,NaN,NaN,0.206872,0.353047,NaN,0.208289,0.188006,0.363489,7.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,51923c5d7573ef46aa9197cae78c3305abea5b3479331f...,6,83.5,NaN,18.0,1.0,0.0,2.5,0.846154,0.150685,0.252103,0.541648,NaN,0.454850,0.480663,0.479467,5.0,60.0
1324,06407d9ec85d62cd38189108ddffec23822f421b3db357...,0,NaN,NaN,20.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
1325,e5015703a58ccd5582a46d9f4a779edf062d683f3ae873...,132,83.0,0.0,20.0,0.0,1.0,0.0,0.961538,0.205479,0.193712,0.533115,0.491958,0.513933,0.508464,0.573437,9.0,132.0
1326,cf6ea2601bb119113371df79931cc3734b77218f734ad0...,12,82.4,NaN,18.0,1.0,0.0,0.5,0.884615,0.205479,0.272259,0.572799,NaN,0.597707,0.380480,0.361608,3.0,12.0


In [8]:
test_B = pd.read_csv(data_dir / "test_B.csv")
test_B = augment_base_dataset(test_B)
test_B[scaled_cols] = scaler.transform(test_B[scaled_cols])

test_B

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
0,90a4f1869cf459af5fe39e53f1c328540f1dcf5a1908f7...,60,67.9,1.0,20.0,0.0,1.0,0.0,1.000000,0.123288,0.069404,NaN,0.330562,NaN,NaN,NaN,6.0,60.0
1,fad8ca8f903cf3ddf566926eabdb8718e8568962675519...,30,69.1,NaN,16.0,0.0,0.0,0.0,0.961538,0.059315,0.162418,0.749086,NaN,0.899471,0.724619,0.481817,3.0,30.0
2,d342fb7689e49c754709870c77e1aa3ed770dd193e9f9c...,12,NaN,NaN,12.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12.0
3,5319e7ba149f0f81715b5e7f854036fc937141840bbd52...,6,NaN,NaN,18.0,NaN,0.0,NaN,NaN,NaN,NaN,0.476637,NaN,0.430159,0.520727,0.415281,5.0,126.0
4,6eef135d8c4eca67b0e130b8f4aedbc37a99938224d661...,0,NaN,0.0,16.0,0.0,0.0,0.0,0.961538,0.095890,NaN,0.605756,0.508509,0.629277,0.604379,0.449383,5.0,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,fbf6267bf7d92b507feb4957d7aa90ea5bb50893bb79d4...,0,NaN,NaN,12.0,NaN,2.0,NaN,NaN,NaN,NaN,0.492325,NaN,0.497354,0.374770,0.460151,7.0,60.0
1466,03e8ddc654f8e27332c5b09618b355d7f9529d614adb0f...,12,81.5,0.0,15.0,NaN,0.0,NaN,NaN,NaN,0.264134,NaN,0.208488,NaN,NaN,NaN,3.0,12.0
1467,1156748dfd6e69e1f364c31584e957d3b1ef656b898942...,0,NaN,0.0,18.0,0.0,0.0,0.0,0.961538,0.136986,NaN,NaN,0.686284,NaN,NaN,NaN,4.0,84.0
1468,0c7e17c442e715e067bd472c1e472b4937914d7fb8d492...,12,NaN,NaN,16.0,NaN,0.0,NaN,NaN,NaN,NaN,0.553883,NaN,0.753086,0.606563,0.458784,4.0,42.0


In [9]:
test_A.isna().sum()

RID_HASH          0
VISCODE           0
AGE             612
PTGENDER_num    626
PTEDUCAT         65
DX_num          428
APOE4            49
CDRSB           428
MMSE            428
ADAS13          428
Ventricles      612
Hippocampus     668
WholeBrain      626
Entorhinal      668
Fusiform        668
MidTemp         668
total_visits      0
last_visit        0
dtype: int64

In [10]:
test_A.columns

Index(['RID_HASH', 'VISCODE', 'AGE', 'PTGENDER_num', 'PTEDUCAT', 'DX_num',
       'APOE4', 'CDRSB', 'MMSE', 'ADAS13', 'Ventricles', 'Hippocampus',
       'WholeBrain', 'Entorhinal', 'Fusiform', 'MidTemp', 'total_visits',
       'last_visit'],
      dtype='object')

In [11]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from hyperimpute.plugins.prediction import Classifiers, Regression
from hyperimpute.utils.tester import evaluate_regression, evaluate_estimator
from hyperimpute.utils.benchmarks import RMSE
from hyperimpute.utils.optimizer import EarlyStoppingExceeded, create_study
import optuna

train_cols = list(dev_set.drop(columns=["RID_HASH"]).columns)

eval_cols = [
    "DX_num",
    "CDRSB",
    "MMSE",
    "ADAS13",
    "Ventricles",
    "Hippocampus",
    "WholeBrain",
    "Entorhinal",
    "Fusiform",
    "MidTemp",
]



def prepare_temporal_data(data, target_col: str, direction: str):
    target_train_data = []
    target_train_labels = []

    for item in data.groupby("RID_HASH"):
        # print(item[0])
        local = item[1]
        local = local.sort_values(["RID_HASH", "VISCODE"])

        rid = local["RID_HASH"]

        prev_cols = [f"prev_{col}" for col in train_cols]
        prev_row = np.zeros(len(prev_cols))

        if direction == "forward":
            rows = local.iterrows()
        else:
            rows = local.iloc[::-1].iterrows()

        for idx, row in rows:
            target_val = row[target_col]
            tmp_row = row[train_cols].copy()
            src_data = tmp_row.to_frame().T.drop(columns=[target_col])

            src_data[prev_cols] = prev_row

            prev_row = tmp_row

            target_train_data.append(src_data)
            target_train_labels.append(target_val)

    target_train_data = pd.concat(target_train_data, ignore_index=True).astype(float)

    return target_train_data, target_train_labels


def evaluate_target(data, target_col: str, direction: str):
    train_data, labels = prepare_temporal_data(data, target_col, direction)
    assert target_col not in train_data.columns

    
    def evaluate_clf(plugin, args = {}):
        model = plugin(**args)
        encoded_labels = LabelEncoder().fit_transform(labels)

        return evaluate_estimator(
            model, train_data, pd.Series(encoded_labels)
        )["clf"]["aucroc"][0]
    
    def evaluate_reg(plugin, args = {}):
        model = plugin(**args)
        return evaluate_regression(model, train_data, labels)["clf"]["r2"][0]

    best_score = -99
    best_target_plugin = None
    for (clf_type, reg_type) in [
        ("lgbm", "lgbm_regressor"),
        ("xgboost", "xgboost_regressor"),
        ("logistic_regression", "linear_regression"),
        ("catboost", "catboost_regressor"),
        ("random_forest", "random_forest_regressor"),
        ("kneighbors", "kneighbors_regressor"),
    ]:
        if len(np.unique(labels)) < cat_limit:
            if clf_type is None:
                continue
            plugin = Classifiers().get_type(clf_type)
            cbk = evaluate_clf
        else:
            if reg_type is None:
                continue
            plugin = Regression().get_type(reg_type)
            cbk = evaluate_reg

        study, pruner = create_study(
            study_name=f"long_imputation_{plugin.name()}_{target_col}_{direction}",
            direction="maximize",
            load_if_exists = True,
        )
        
        def objective(trial: optuna.Trial) -> float:
            args = plugin.sample_hyperparameters(trial)
            pruner.check_trial(trial)

            try:
                score = cbk(plugin, args)
            except BaseException:
                print("      failed evaluation", plugin.name(), args)
                return -5

            #print(f"    >>  {plugin.name()} {args} -> {score}")
            pruner.report_score(score)

            return score

        try:
            study.optimize(objective, n_trials=100, timeout=60 * 10)
        except EarlyStoppingExceeded:
            pass

        baseline_score = cbk(plugin)

        if study.best_value > baseline_score:
            score = study.best_value
            args = study.best_trial.params
        else:
            score = baseline_score
            args = {}
            
        if score > best_score:
            best_score = score
            best_target_plugin, best_target_plugin_args = plugin, args
            
    print(f"     >> Selected {target_col} --> {best_target_plugin.name()} -- {best_target_plugin_args}", best_score)

    model = best_target_plugin(**best_target_plugin_args)
        
    if len(np.unique(labels)) < cat_limit:
        labels = LabelEncoder().fit_transform(labels)
    
    model.fit(train_data, labels)
    
    return model


def prepare_longitudinal_imputers(data, columns):
    imputers = {}

    for direction in ["forward", "reverse"]:
        imputers[direction] = {}
        for target_col in columns:
            train_data, labels = prepare_temporal_data(data, target_col, direction)
            print("train", target_col, direction, len(np.unique(labels)))

            if len(np.unique(labels)) > cat_limit:
                model = Regression().get("catboost_regressor")
            else:
                model = Classifiers().get("catboost")

            model.fit(train_data, labels)

            imputers[direction][target_col] = model

    return imputers


def prepare_longitudinal_imputers_automl(data, columns):
    imputers = {}

    for direction in ["forward", "reverse"]:
        imputers[direction] = {}
        for target_col in columns:
            model = evaluate_target(data, target_col, direction=direction)

            imputers[direction][target_col] = model

    return imputers

In [ ]:
for col in eval_cols:
    evaluate_target(dev_set, col, direction = "forward")

     >> Selected DX_num --> xgboost -- {'reg_lambda': 8.962330859372681, 'reg_alpha': 8.608303558298164, 'colsample_bytree': 0.7745875968255177, 'colsample_bynode': 0.3990200802902815, 'colsample_bylevel': 0.8924327091878707, 'subsample': 0.8411533780703977, 'lr': 0.0001, 'max_depth': 3, 'n_estimators': 169, 'min_child_weight': 1, 'max_bin': 256, 'booster': 1} 0.9877047491583673
     >> Selected CDRSB --> linear_regression -- {'max_iter': 10000, 'solver': 1} 0.8235423012308886
     >> Selected MMSE --> xgboost_regressor -- {'reg_lambda': 2.1888371329742604, 'reg_alpha': 0.9312430343555846, 'max_depth': 2, 'n_estimators': 82, 'lr': 0.01} 0.7464127570286078
     >> Selected ADAS13 --> linear_regression -- {} 0.8184916603842121
     >> Selected Ventricles --> linear_regression -- {'max_iter': 100, 'solver': 1} 0.7916836168497275


In [ ]:
for col in eval_cols:
    evaluate_target(dev_set, col, direction = "reverse")

In [ ]:
raise

In [16]:
dev_set_id = dataframe_hash(dev_set)

imputers_bkp_file = workspace / f"longitudinal_imputers_scaled_cat{cat_limit}_{dev_set_id}_automl_100.bkp"

if imputers_bkp_file.exists():
    longitudinal_imputers = load_model_from_file(imputers_bkp_file)
else:
    longitudinal_imputers = prepare_longitudinal_imputers_automl(dev_set, eval_cols)
    save_model_to_file(imputers_bkp_file, longitudinal_imputers)

In [17]:
for direction in longitudinal_imputers:
    for col in longitudinal_imputers[direction]:
        print(direction, col,  longitudinal_imputers[direction][col].name())

forward DX_num xgboost
forward CDRSB linear_regression
forward MMSE xgboost_regressor
forward ADAS13 linear_regression
forward Ventricles linear_regression
forward Hippocampus xgboost_regressor
forward WholeBrain linear_regression
forward Entorhinal linear_regression
forward Fusiform linear_regression
forward MidTemp linear_regression
reverse DX_num xgboost
reverse CDRSB xgboost_regressor
reverse MMSE xgboost_regressor
reverse ADAS13 xgboost_regressor
reverse Ventricles linear_regression
reverse Hippocampus xgboost_regressor
reverse WholeBrain linear_regression
reverse Entorhinal linear_regression
reverse Fusiform linear_regression
reverse MidTemp linear_regression


## Preprocess data

In [18]:
from hyperimpute.plugins.imputers import Imputers

# VISCODE 6 * x -> AGE 0.5 * x

const_by_patient = ["PTGENDER_num", "PTEDUCAT", "APOE4"]


def dataframe_hash(df: pd.DataFrame) -> str:
    return str(abs(pd.util.hash_pandas_object(df).sum()))


def normalize(test_data):
    return test_data

def prepare_consts(train_data, test_data):
    test_data = test_data.copy()
    train_data = train_data.copy()

    train_data = train_data.sort_values(["RID_HASH", "VISCODE"])
    test_data = test_data.sort_values(["RID_HASH", "VISCODE"])

    for item in test_data.groupby("RID_HASH"):
        local = item[1]

        # fill consts
        for col in const_by_patient:
            if len(local[col].unique()) == 1:
                continue
            rid = local["RID_HASH"].unique()[0]

            val = local[col][~local[col].isna()].unique()[0]
            local[col] = local[col].fillna(val)
            test_data.loc[test_data["RID_HASH"] == rid, col] = test_data[
                test_data["RID_HASH"] == rid
            ][col].fillna(val)
            assert len(local[col].unique()) == 1, col

    return test_data


def prepare_age(train_data, test_data):
    test_data = test_data.copy()
    train_data = train_data.copy()

    train_data = train_data.sort_values(["RID_HASH", "VISCODE"])
    test_data = test_data.sort_values(["RID_HASH", "VISCODE"])

    col = "AGE"

    for rid in test_data["RID_HASH"].unique():
        local = test_data[test_data["RID_HASH"] == rid]

        # fill age
        ages = local["AGE"]
        if ages.isna().sum() == 0:
            continue

        if ages.isna().sum() == len(ages):
            continue

        # forward impute age
        prev_viscode = 0
        prev_age = 0
        for idx, row in local.iterrows():
            current_viscode = row["VISCODE"]
            local_idx = (test_data["VISCODE"] == current_viscode) & (
                test_data["RID_HASH"] == rid
            )
            if prev_age > 0 and prev_age == prev_age:
                pred_age = (current_viscode - prev_viscode) / 6 * 0.5 + prev_age
            else:
                pred_age = row[col]

            if pred_age == pred_age:
                # print("forward imputed", pred_age, current_viscode)
                test_data.loc[local_idx, col] = test_data.loc[local_idx][col].fillna(
                    pred_age
                )

            prev_viscode = row["VISCODE"]
            prev_age = pred_age

        # reverse impute age
        prev_viscode = 0
        prev_age = 0
        for idx, row in local.iloc[::-1].iterrows():
            current_viscode = row["VISCODE"]
            local_idx = (test_data["VISCODE"] == current_viscode) & (
                test_data["RID_HASH"] == rid
            )

            if prev_age > 0 and prev_age == prev_age:
                pred_age = prev_age - (prev_viscode - current_viscode) / 6 * 0.5
            else:
                pred_age = row[col]

            if pred_age == pred_age:
                # print("reversed imputed", pred_age, current_viscode)
                test_data.loc[local_idx, col] = test_data.loc[local_idx][col].fillna(
                    pred_age
                )

            prev_viscode = row["VISCODE"]
            prev_age = pred_age

        # print(test_data[(test_data["RID_HASH"] == rid)][["VISCODE", "AGE"]])
    return test_data


def impute_longitudinal(
    train_data,
    test_data,
    n_iter=5,
    eval_cols=[
        "DX_num",
        "CDRSB",
        "MMSE",
        "ADAS13",
        "Ventricles",
        "Hippocampus",
        "WholeBrain",
        "Entorhinal",
        "Fusiform",
        "MidTemp",
    ],
    imputed_test_data=None,
):
    test_data = test_data.copy()
    train_data = train_data.copy()

    if imputed_test_data is None:
        imputed_test_data = interm_imputation(
            train_data, test_data
        )

    train_data = train_data.sort_values(["RID_HASH", "VISCODE"])
    test_data = test_data.sort_values(["RID_HASH", "VISCODE"])
    imputed_test_data = imputed_test_data.sort_values(["RID_HASH", "VISCODE"])

    prev_cols = [f"prev_{col}" for col in train_cols]

    for rid in test_data["RID_HASH"].unique():
        patient = test_data[test_data["RID_HASH"] == rid]
        patient_imputed = imputed_test_data[imputed_test_data["RID_HASH"] == rid]

        prediction_rows = [pd.Series(np.zeros(len(prev_cols)), index=train_cols)]
        for ridx, row in patient.iterrows():
            prediction_rows.append(row[train_cols])
        prediction_rows.append(pd.Series(np.zeros(len(prev_cols)), index=train_cols))

        for col in eval_cols:
            if patient[col].isna().sum() == 0:
                continue

            for ridx, row in enumerate(prediction_rows[1:-1]):
                real_idx = ridx + 1
                if row[col] == row[col]:
                    continue
                current_viscode = row["VISCODE"]
                local_idx = (test_data["VISCODE"] == current_viscode) & (
                    test_data["RID_HASH"] == rid
                )

                prev_col_val = prediction_rows[real_idx - 1][col]
                next_col_val = prediction_rows[real_idx + 1][col]

                if next_col_val == next_col_val and ridx + 1 < len(patient_imputed):
                    eval_data = (
                        patient_imputed.iloc[ridx].to_frame().T[train_cols]
                    ).drop(columns = [col]) #row.to_frame().T[train_cols]
                    eval_data[prev_cols] = (
                        patient_imputed.iloc[ridx + 1].to_frame().T[train_cols].values
                    )
                    eval_data = eval_data.astype(float)

                    assert eval_data.isna().sum().sum() == 0
                    assert eval_data[f"prev_{col}"].values[0] == next_col_val

                    imputer = longitudinal_imputers["reverse"][col]
                    imputed_val = imputer.predict(eval_data).values.squeeze()

                    test_data.loc[local_idx, col] = imputed_val

                if prev_col_val == prev_col_val and ridx > 0:
                    # print("Imputing using the prev value", prev_col_val)
                    eval_data = (
                        patient_imputed.iloc[ridx].to_frame().T[train_cols]
                    ).drop(columns = [col])
                    eval_data[prev_cols] = (
                        patient_imputed.iloc[ridx - 1].to_frame().T[train_cols].values
                    )
                    eval_data = eval_data.astype(float)

                    assert eval_data.isna().sum().sum() == 0
                    assert eval_data[f"prev_{col}"].values[0] == prev_col_val

                    imputer = longitudinal_imputers["forward"][col]
                    imputed_val = imputer.predict(eval_data).values.squeeze()
                    
                    existing_value = test_data.loc[local_idx, col].values[0]
                    if existing_value == existing_value:
                        imputed_val = (imputed_val + existing_value) / 2
                    test_data.loc[local_idx, col] = imputed_val


    return normalize(test_data)

def interm_imputation(train_data, test_data, forward_first = True):
    test_data = test_data.copy()

    for rid in test_data["RID_HASH"].unique():
        local = test_data[test_data["RID_HASH"] == rid]

        if forward_first:
            local = local.ffill().bfill()
        else:
            local = local.bfill().ffill()

        test_data.loc[test_data["RID_HASH"] == rid] = local
        
    test_data = prepare_consts(train_data, test_data)
    test_data = prepare_age(train_data, test_data)
    return full_imputation(train_data, test_data)

def full_imputation(train_data, test_data):
    imputed_test_data = test_data.copy()
    imputer_kwargs = {
        "optimizer": "bayesian",
        "classifier_seed": ["xgboost",],
        "regression_seed": ["xgboost_regressor"],
        "class_threshold": cat_limit,
    }

    imputer = Imputers().get(
        "hyperimpute",
        **imputer_kwargs,
    )
    imputation_input = pd.concat([train_data, test_data], ignore_index=True)
    imputation_ids = imputation_input["RID_HASH"]
    imputation_input = imputation_input.drop(columns = ["RID_HASH"])
    
    imputed_test_data = imputer.fit_transform(imputation_input)
    imputed_test_data = imputed_test_data.tail(len(test_data))

    out_cols = ["RID_HASH"] + list(imputed_test_data.columns)
    imputed_test_data["RID_HASH"] = test_data["RID_HASH"].values
    
    return imputed_test_data[out_cols]

def evaluate_static_imputation(train_data, test_data, static_imputation):
    test_data = test_data.copy()
    train_data = train_data.copy()

    train_data = train_data.sort_values(["RID_HASH", "VISCODE"])
    test_data = test_data.sort_values(["RID_HASH", "VISCODE"])

    for rid in test_data["RID_HASH"].unique():
        patient = test_data[test_data["RID_HASH"] == rid]
        misses = []
        viscodes = []
        for idx, row in patient.iterrows():
            misses.append(row.isna().sum())
            viscodes.append(row["VISCODE"])
        cidx = np.argmin(misses)

        current_viscode = viscodes[cidx]
        local_idx = (test_data["VISCODE"] == current_viscode) & (
            test_data["RID_HASH"] == rid
        )
        imputed_idx = (static_imputation["VISCODE"] == current_viscode) & (
            static_imputation["RID_HASH"] == rid
        )

        if len(test_data[local_idx]) == 0:
            continue

        for col in test_data.columns:
            val = test_data.loc[local_idx][col].values[0]
            if val == val:
                continue
            imputed_val = static_imputation.loc[imputed_idx][col].values[0]
            test_data.loc[local_idx, col] = imputed_val

            # print("imputed", test_data.loc[local_idx, col])

    return normalize(test_data)


def impute_data(
    train_data, test_data, use_longitudinal=True, static_strategy="missmin"
):
    test_id = dataframe_hash(test_data)
    train_id = dataframe_hash(train_data)

    print("Evaluate constants", test_id, test_data.isna().sum().sum())
    test_data = prepare_consts(train_data, test_data)
    test_data = prepare_age(train_data, test_data)

    while use_longitudinal:
        print("Evaluate longitudinals", test_id, test_data.isna().sum().sum())
        new_test_data = impute_longitudinal(train_data, test_data)
        if new_test_data.isna().sum().sum() == test_data.isna().sum().sum():
            break

        test_data = new_test_data

    print(
        "Evaluate static imputation",
        test_id,
        test_data.isna().sum().sum(),
        static_strategy,
    )
    static_imputation = full_imputation(train_data, test_data)

    test_data = evaluate_static_imputation(train_data, test_data, static_imputation)

    print("Evaluate constants take 2", test_id, test_data.isna().sum().sum())
    test_data = prepare_consts(train_data, test_data)
    test_data = prepare_age(train_data, test_data)

    while use_longitudinal:
        print("Evaluate longitudinals take 2", test_id, test_data.isna().sum().sum())
        new_test_data = impute_longitudinal(train_data, test_data)
        if new_test_data.isna().sum().sum() == test_data.isna().sum().sum():
            break

        test_data = new_test_data

    print("Normalize data", test_id, test_data.isna().sum().sum())
    return normalize(test_data)

In [19]:
dev_1_eval = impute_data(dev_set, dev_1)
dev_2_eval = impute_data(dev_set, dev_2)

Evaluate constants 8756985925945686816 22466
Evaluate longitudinals 8756985925945686816 19844


[2022-11-09T12:26:03.494688+0200][3193248][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: False
[2022-11-09T12:26:03.521989+0200][3193248][INFO]   > HyperImpute using inner optimization
[2022-11-09T12:26:03.522620+0200][3193248][INFO]   > Imputation iter 0
[2022-11-09T12:26:03.725939+0200][3193248][INFO] [Evaluate PTGENDER_num] previous config new score = 0.9352865489811768. old score = -9999999
[2022-11-09T12:26:05.323179+0200][3193248][INFO]      >>> Column PTGENDER_num <-- score 0.9352865489811768 <-- Model xgboost({})
[2022-11-09T12:26:05.690715+0200][3193248][INFO] [Evaluate MidTemp] previous config new score = 0.7208799560547148. old score = -9999999
[2022-11-09T12:26:06.762308+0200][3193248][INFO]      >>> Column MidTemp <-- score 0.7208799560547148 <-- Model xgboost_regressor({})
[2022-11-09T12:26:07.123063+0200][3193248][INFO] [Evaluate PTEDUCAT] previous config new score = 0.3383687029319891. old score = -9999999
[2022-11-09T12:26:26.9092

Evaluate longitudinals 8756985925945686816 14795


[2022-11-09T12:30:54.959441+0200][3193248][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: False
[2022-11-09T12:30:54.983424+0200][3193248][INFO]   > HyperImpute using inner optimization
[2022-11-09T12:30:54.983991+0200][3193248][INFO]   > Imputation iter 0
[2022-11-09T12:30:55.186054+0200][3193248][INFO] [Evaluate PTGENDER_num] previous config new score = 0.9311935776506117. old score = -9999999
[2022-11-09T12:30:56.774330+0200][3193248][INFO]      >>> Column PTGENDER_num <-- score 0.9311935776506117 <-- Model xgboost({})
[2022-11-09T12:30:57.150268+0200][3193248][INFO] [Evaluate MidTemp] previous config new score = 0.7253432868938026. old score = -9999999
[2022-11-09T12:30:58.164353+0200][3193248][INFO]      >>> Column MidTemp <-- score 0.7253432868938026 <-- Model xgboost_regressor({})
[2022-11-09T12:30:58.534577+0200][3193248][INFO] [Evaluate PTEDUCAT] previous config new score = 0.31352164412351885. old score = -9999999
[2022-11-09T12:30:59.895

Evaluate longitudinals 8756985925945686816 11519


...

Evaluate longitudinals take 2 1637477679073542181 1


[2022-11-09T13:50:06.752330+0200][3193248][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: False
[2022-11-09T13:50:06.776781+0200][3193248][INFO]   > HyperImpute using inner optimization
[2022-11-09T13:50:06.777399+0200][3193248][INFO]   > Imputation iter 0
[2022-11-09T13:50:06.785085+0200][3193248][INFO]   > Imputation iter 1
[2022-11-09T13:50:06.791909+0200][3193248][INFO]   > Imputation iter 2
[2022-11-09T13:50:06.798483+0200][3193248][INFO]   > Imputation iter 3
[2022-11-09T13:50:06.805076+0200][3193248][INFO]   > Imputation iter 4
[2022-11-09T13:50:06.811694+0200][3193248][INFO]   > Imputation iter 5
[2022-11-09T13:50:06.818437+0200][3193248][INFO]   > Imputation iter 6
[2022-11-09T13:50:06.825541+0200][3193248][INFO]   > Imputation iter 7
[2022-11-09T13:50:06.832229+0200][3193248][INFO]   > Imputation iter 8
[2022-11-09T13:50:06.839138+0200][3193248][INFO]   > Imputation iter 9
[2022-11-09T13:50:06.845872+0200][3193248][INFO]   > Imputation it

Evaluate longitudinals take 2 1637477679073542181 0


[2022-11-09T13:50:16.861232+0200][3193248][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: False
[2022-11-09T13:50:16.886701+0200][3193248][INFO]   > HyperImpute using inner optimization
[2022-11-09T13:50:16.887296+0200][3193248][INFO]   > Imputation iter 0
[2022-11-09T13:50:16.894875+0200][3193248][INFO]   > Imputation iter 1
[2022-11-09T13:50:16.901547+0200][3193248][INFO]   > Imputation iter 2
[2022-11-09T13:50:16.908129+0200][3193248][INFO]   > Imputation iter 3
[2022-11-09T13:50:16.915021+0200][3193248][INFO]   > Imputation iter 4
[2022-11-09T13:50:16.922110+0200][3193248][INFO]   > Imputation iter 5
[2022-11-09T13:50:16.929063+0200][3193248][INFO]   > Imputation iter 6
[2022-11-09T13:50:16.935898+0200][3193248][INFO]   > Imputation iter 7
[2022-11-09T13:50:16.942752+0200][3193248][INFO]   > Imputation iter 8
[2022-11-09T13:50:16.949459+0200][3193248][INFO]   > Imputation iter 9
[2022-11-09T13:50:16.956050+0200][3193248][INFO]   > Imputation it

Normalize data 1637477679073542181 0


In [20]:
# use static missmin visit strategy
from hyperimpute.plugins.imputers import Imputers
from hyperimpute.utils.benchmarks import benchmark_model
from sklearn.preprocessing import LabelEncoder

train_eval_data = pd.concat([dev_set, dev_1_eval, dev_2_eval], ignore_index=True)

train_eval_data_raw = pd.concat([dev_set, dev_1, dev_2], ignore_index=True)

train_gt = pd.concat([dev_set, dev_set, dev_set], ignore_index=True)

train_mask = train_eval_data_raw.isna().astype(int)
train_mask_bool = train_eval_data_raw.isna()

le = LabelEncoder().fit(train_gt["RID_HASH"])
train_gt["RID_HASH"] = le.transform(train_gt["RID_HASH"])
train_eval_data["RID_HASH"] = le.transform(train_eval_data["RID_HASH"])

plugin = Imputers().get(
    "hyperimpute",
    optimizer="simple",
)

benchmark_model("missforest", plugin, train_gt, train_eval_data, train_mask)

[2022-11-09T13:50:21.640550+0200][3193248][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: False
[2022-11-09T13:50:21.673222+0200][3193248][INFO]   > HyperImpute using inner optimization
[2022-11-09T13:50:21.673899+0200][3193248][INFO]   > Imputation iter 0
[2022-11-09T13:50:21.683617+0200][3193248][INFO]   > Imputation iter 1
[2022-11-09T13:50:21.692370+0200][3193248][INFO]   > Imputation iter 2
[2022-11-09T13:50:21.700994+0200][3193248][INFO]   > Imputation iter 3
[2022-11-09T13:50:21.709919+0200][3193248][INFO]   > Imputation iter 4
[2022-11-09T13:50:21.718332+0200][3193248][INFO]   > Imputation iter 5
[2022-11-09T13:50:21.726671+0200][3193248][INFO]   > Imputation iter 6
[2022-11-09T13:50:21.735120+0200][3193248][INFO]   > Imputation iter 7
[2022-11-09T13:50:21.743636+0200][3193248][INFO]   > Imputation iter 8
[2022-11-09T13:50:21.752135+0200][3193248][INFO]   > Imputation iter 9
[2022-11-09T13:50:21.760754+0200][3193248][INFO]   > Imputation it

(1.2126093403476297, 0.4807779568291668)

In [21]:
# Current best (1.2941082455707285, 0.6173876160675963)


## Submission data

In [22]:
test_A_eval = impute_data(dev_set, test_A)
test_B_eval = impute_data(dev_set, test_B)

Evaluate constants 1708900434718589799 6974
Evaluate longitudinals 1708900434718589799 6161


[2022-11-09T13:50:24.937492+0200][3193248][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: False
[2022-11-09T13:50:24.959505+0200][3193248][INFO]   > HyperImpute using inner optimization
[2022-11-09T13:50:24.960056+0200][3193248][INFO]   > Imputation iter 0
[2022-11-09T13:50:25.171800+0200][3193248][INFO] [Evaluate PTGENDER_num] previous config new score = 0.9110322380000676. old score = -9999999
[2022-11-09T13:50:26.752141+0200][3193248][INFO]      >>> Column PTGENDER_num <-- score 0.9110322380000676 <-- Model xgboost({})
[2022-11-09T13:50:27.089756+0200][3193248][INFO] [Evaluate MidTemp] previous config new score = 0.6850865995975314. old score = -9999999
[2022-11-09T13:50:28.096097+0200][3193248][INFO]      >>> Column MidTemp <-- score 0.6850865995975314 <-- Model xgboost_regressor({})
[2022-11-09T13:50:28.435177+0200][3193248][INFO] [Evaluate PTEDUCAT] previous config new score = 0.24010931618225062. old score = -9999999
[2022-11-09T13:50:48.391

Evaluate longitudinals 1708900434718589799 4562


[2022-11-09T13:54:15.670136+0200][3193248][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: False
[2022-11-09T13:54:15.691748+0200][3193248][INFO]   > HyperImpute using inner optimization
[2022-11-09T13:54:15.692300+0200][3193248][INFO]   > Imputation iter 0
[2022-11-09T13:54:15.912228+0200][3193248][INFO] [Evaluate PTGENDER_num] previous config new score = 0.9140756219543197. old score = -9999999
[2022-11-09T13:54:17.488728+0200][3193248][INFO]      >>> Column PTGENDER_num <-- score 0.9140756219543197 <-- Model xgboost({})
[2022-11-09T13:54:17.828643+0200][3193248][INFO] [Evaluate MidTemp] previous config new score = 0.6884131956602488. old score = -9999999
[2022-11-09T13:54:19.026115+0200][3193248][INFO]      >>> Column MidTemp <-- score 0.6884131956602488 <-- Model xgboost_regressor({})
[2022-11-09T13:54:19.363455+0200][3193248][INFO] [Evaluate PTEDUCAT] previous config new score = 0.22340113433057002. old score = -9999999
[2022-11-09T13:54:40.265

Evaluate longitudinals 1708900434718589799 3497


...

Evaluate longitudinals take 2 8191117053172621229 4


[2022-11-09T14:56:22.902011+0200][3193248][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: False
[2022-11-09T14:56:22.924844+0200][3193248][INFO]   > HyperImpute using inner optimization
[2022-11-09T14:56:22.925467+0200][3193248][INFO]   > Imputation iter 0
[2022-11-09T14:56:22.932060+0200][3193248][INFO]   > Imputation iter 1
[2022-11-09T14:56:22.937962+0200][3193248][INFO]   > Imputation iter 2
[2022-11-09T14:56:22.944017+0200][3193248][INFO]   > Imputation iter 3
[2022-11-09T14:56:22.949710+0200][3193248][INFO]   > Imputation iter 4
[2022-11-09T14:56:22.955443+0200][3193248][INFO]   > Imputation iter 5
[2022-11-09T14:56:22.961134+0200][3193248][INFO]   > Imputation iter 6
[2022-11-09T14:56:22.966906+0200][3193248][INFO]   > Imputation iter 7
[2022-11-09T14:56:22.972644+0200][3193248][INFO]   > Imputation iter 8
[2022-11-09T14:56:22.978342+0200][3193248][INFO]   > Imputation iter 9
[2022-11-09T14:56:22.984043+0200][3193248][INFO]   > Imputation it

Evaluate longitudinals take 2 8191117053172621229 0


[2022-11-09T14:56:26.176331+0200][3193248][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: False
[2022-11-09T14:56:26.198313+0200][3193248][INFO]   > HyperImpute using inner optimization
[2022-11-09T14:56:26.198851+0200][3193248][INFO]   > Imputation iter 0
[2022-11-09T14:56:26.205442+0200][3193248][INFO]   > Imputation iter 1
[2022-11-09T14:56:26.211280+0200][3193248][INFO]   > Imputation iter 2
[2022-11-09T14:56:26.217309+0200][3193248][INFO]   > Imputation iter 3
[2022-11-09T14:56:26.223347+0200][3193248][INFO]   > Imputation iter 4
[2022-11-09T14:56:26.229349+0200][3193248][INFO]   > Imputation iter 5
[2022-11-09T14:56:26.235389+0200][3193248][INFO]   > Imputation iter 6
[2022-11-09T14:56:26.241295+0200][3193248][INFO]   > Imputation iter 7
[2022-11-09T14:56:26.247065+0200][3193248][INFO]   > Imputation iter 8
[2022-11-09T14:56:26.252860+0200][3193248][INFO]   > Imputation iter 9
[2022-11-09T14:56:26.258520+0200][3193248][INFO]   > Imputation it

Normalize data 8191117053172621229 0


In [23]:
eval_data = pd.concat([dev_set, test_A_eval, test_B_eval], ignore_index=True)


eval_data

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
0,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,0,79.100000,0.0,20.0,1.0,1.0,0.5,0.923077,0.164384,0.071871,0.548646,0.376516,0.464021,0.194906,0.400709,2.0,6.0
1,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,6,79.600000,0.0,20.0,1.0,1.0,1.5,0.923077,0.237397,0.071956,0.548307,0.366398,0.403880,0.193367,0.397291,2.0,6.0
2,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,0,72.900000,1.0,12.0,1.0,1.0,1.0,1.000000,0.123288,0.142655,0.525169,0.235599,0.513404,0.356253,0.294774,6.0,60.0
3,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,6,73.400000,1.0,12.0,1.0,1.0,1.0,1.000000,0.164384,0.144729,0.549210,0.230361,0.435097,0.322395,0.294175,6.0,60.0
4,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,12,73.900000,1.0,12.0,1.0,1.0,1.0,0.961538,0.109589,0.155550,0.527878,0.215944,0.487831,0.342600,0.277552,6.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6894,ff4eb5a64e2b89861d5dea81190669893070b227f3a335...,0,74.304489,0.0,18.0,1.0,1.0,1.5,0.884615,0.114110,0.322319,0.502370,0.492262,0.394356,0.397160,0.531003,3.0,84.0
6895,ff4eb5a64e2b89861d5dea81190669893070b227f3a335...,12,75.304489,0.0,18.0,1.0,1.0,1.5,0.923077,0.242055,0.339084,0.519639,0.509578,0.294356,0.416522,0.545575,3.0,84.0
6896,ff4eb5a64e2b89861d5dea81190669893070b227f3a335...,84,81.304489,0.0,18.0,1.0,1.0,1.5,1.000000,0.178082,0.412867,0.432054,0.483387,0.363316,0.468451,0.508440,3.0,84.0
6897,ffa86109ba8684f31325842d0ff26568e105f0f63b366a...,0,66.300000,1.0,13.0,0.0,0.0,0.0,0.923077,0.118767,0.177669,0.603072,0.335950,0.536760,0.308488,0.445137,2.0,24.0


In [24]:
from hyperimpute.plugins.imputers import Imputers

plugin = Imputers().get(
    "hyperimpute",
    optimizer="simple",
    classifier_seed=["catboost"],
    regression_seed=["xgboost_regressor", "catboost_regressor"],
    class_threshold=cat_limit,
)


imputed_X = plugin.fit_transform(eval_data.copy())
imputed_X[scaled_cols] = scaler.inverse_transform(imputed_X[scaled_cols])

imputed_X

[2022-11-09T14:56:27.829144+0200][3193248][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: False
[2022-11-09T14:56:27.864814+0200][3193248][INFO]   > HyperImpute using inner optimization
[2022-11-09T14:56:27.865434+0200][3193248][INFO]   > Imputation iter 0
[2022-11-09T14:56:27.873042+0200][3193248][INFO]   > Imputation iter 1
[2022-11-09T14:56:27.879696+0200][3193248][INFO]   > Imputation iter 2
[2022-11-09T14:56:27.886314+0200][3193248][INFO]   > Imputation iter 3
[2022-11-09T14:56:27.892947+0200][3193248][INFO]   > Imputation iter 4
[2022-11-09T14:56:27.899744+0200][3193248][INFO]   > Imputation iter 5
[2022-11-09T14:56:27.906297+0200][3193248][INFO]   > Imputation iter 6
[2022-11-09T14:56:27.913054+0200][3193248][INFO]   > Imputation iter 7
[2022-11-09T14:56:27.919666+0200][3193248][INFO]   > Imputation iter 8
[2022-11-09T14:56:27.926695+0200][3193248][INFO]   > Imputation iter 9
[2022-11-09T14:56:27.933449+0200][3193248][INFO]   > Imputation it

,RID_HASH,VISCODE,AGE,PTGENDER_num,PTEDUCAT,DX_num,APOE4,CDRSB,MMSE,ADAS13,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,total_visits,last_visit
0,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,0.0,79.100000,0.0,20.0,1.0,1.0,0.5,28.0,12.00,16636.000000,7208.000000,9.790100e+05,3672.000000,12661.000000,18165.000000,2.0,6.0
1,001c7955017f905ccf78d55c94e81070a1cca7b1efb5bd...,6.0,79.600000,0.0,20.0,1.0,1.0,1.5,28.0,17.33,16649.000000,7205.000000,9.707820e+05,3331.000000,12630.000000,18085.000000,2.0,6.0
2,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,0.0,72.900000,1.0,12.0,1.0,1.0,1.0,30.0,9.00,27456.000000,7000.000000,8.644140e+05,3952.000000,15911.000000,15686.000000,6.0,60.0
3,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,6.0,73.400000,1.0,12.0,1.0,1.0,1.0,30.0,12.00,27773.000000,7213.000000,8.601540e+05,3508.000000,15229.000000,15672.000000,6.0,60.0
4,00e6fb56250581a8c8b5133f91443dd8c037e3cd8d0ba8...,12.0,73.900000,1.0,12.0,1.0,1.0,1.0,29.0,8.00,29427.000000,7024.000000,8.484300e+05,3807.000000,15636.000000,15283.000000,6.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6894,ff4eb5a64e2b89861d5dea81190669893070b227f3a335...,0.0,74.304489,0.0,18.0,1.0,1.0,1.5,27.0,8.33,54919.015071,6798.000000,1.073138e+06,3277.000000,16735.000000,21214.000000,3.0,84.0
6895,ff4eb5a64e2b89861d5dea81190669893070b227f3a335...,12.0,75.304489,0.0,18.0,1.0,1.0,1.5,28.0,17.67,57481.678221,6951.000000,1.087219e+06,2710.000000,17125.000000,21555.000000,3.0,84.0
6896,ff4eb5a64e2b89861d5dea81190669893070b227f3a335...,84.0,81.304489,0.0,18.0,1.0,1.0,1.5,30.0,13.00,68759.948410,6175.000000,1.065920e+06,3101.000000,18171.000000,20686.000000,3.0,84.0
6897,ffa86109ba8684f31325842d0ff26568e105f0f63b366a...,0.0,66.300000,1.0,13.0,0.0,0.0,0.0,28.0,8.67,32808.200000,7690.220979,9.460212e+05,4084.430357,14948.865953,19204.643123,2.0,24.0


In [27]:
import numpy as np

results = []


def normalize_output(test_data):
    test_data = test_data.copy()

    factor = test_data["CDRSB"] / 0.5
    factor[factor < 0] = 0
    factor = factor.fillna(-1)
    factor = factor.round(0).astype(int)
    factor = factor.replace(-1, np.nan)
    test_data["CDRSB"] = factor * 0.5
    
    test_data["DX_num"] = test_data["DX_num"].round(0)

    test_data["ADAS13"] = ((test_data["ADAS13"] * 3).round(0) / 3).round(2)
    test_data["MMSE"] = test_data["MMSE"].round(0)

    return test_data

def dump_results(imputed_data: pd.DataFrame, fpath: str):
    for name, data in [
        ("test_A", test_A),
        ("test_B", test_B),
    ]:
        for idx, row in data.iterrows():
            for col in row.index:
                local = row.T
                val = local[col]
                if val == val:
                    continue
                imputed_id = f"{local['RID_HASH']}_{local['VISCODE']}_{col}_{name}"
                imputed_val = imputed_data[
                    (imputed_data["RID_HASH"] == local["RID_HASH"])
                    & (imputed_data["VISCODE"] == local["VISCODE"])
                ][col].values[0]
                
                assert imputed_val == imputed_val
                assert imputed_val != ""
                
                results.append([imputed_id, imputed_val])

    output = pd.DataFrame(results, columns=submission.columns)
    output.to_csv(fpath, index=None)

    return output


version = "v9"
changelog = "automl_one_more_try"
output_normalized = dump_results(
    normalize_output(imputed_X),
    results_dir / f"imputation_results_{version}_{changelog}.csv",
)

output_normalized

,Id,Predicted
0,988b6137f4352c01e4b52790505caa0c3ec438f117000a...,58.20
1,988b6137f4352c01e4b52790505caa0c3ec438f117000a...,0.00
2,988b6137f4352c01e4b52790505caa0c3ec438f117000a...,2.00
3,988b6137f4352c01e4b52790505caa0c3ec438f117000a...,3.50
4,988b6137f4352c01e4b52790505caa0c3ec438f117000a...,26.00
...,...,...
15021,8b33cc9dd06fc18f130e185fdf1e6d657dbc80add9ff6e...,1.00
15022,8b33cc9dd06fc18f130e185fdf1e6d657dbc80add9ff6e...,0.00
15023,8b33cc9dd06fc18f130e185fdf1e6d657dbc80add9ff6e...,1.00
15024,8b33cc9dd06fc18f130e185fdf1e6d657dbc80add9ff6e...,29.00


In [28]:
pd.set_option("display.expand_frame_repr", True)

output_normalized.tail(5).values

array([['8b33cc9dd06fc18f130e185fdf1e6d657dbc80add9ff6ea81ced3e328cea9e63_0_DX_num_test_B',
        1.0],
       ['8b33cc9dd06fc18f130e185fdf1e6d657dbc80add9ff6ea81ced3e328cea9e63_0_APOE4_test_B',
        0.0],
       ['8b33cc9dd06fc18f130e185fdf1e6d657dbc80add9ff6ea81ced3e328cea9e63_0_CDRSB_test_B',
        1.0],
       ['8b33cc9dd06fc18f130e185fdf1e6d657dbc80add9ff6ea81ced3e328cea9e63_0_MMSE_test_B',
        29.0],
       ['8b33cc9dd06fc18f130e185fdf1e6d657dbc80add9ff6ea81ced3e328cea9e63_0_ADAS13_test_B',
        18.33]], dtype=object)